In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from datasets import Dataset
import os

/home/siddhi/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/siddhi/miniconda3/envs/towerbase-qlora/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/siddhi/miniconda3/envs/towerbase-qlora/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/siddhi/miniconda3/envs/towerbase-qlora/lib/python3.10/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soo

In [ ]:
model_id = "/data3/ritika_project/mistral-7b"


new_model_name = "mistral-7b-english-to-french"
output_base_dir = "/data3/ritika/mistral_training_output"

os.makedirs(output_base_dir, exist_ok=True)

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

print(f"Loading base model from: {model_id}")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" 
print("Base model loaded.")

Loading base model from: /data3/ritika_project/mistral-7b


Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████| 2/2 [00:08<00:00,  4.26s/it]


Base model loaded.


In [4]:
model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
print("Model prepared for QLoRA training.")

Model prepared for QLoRA training.


In [ ]:
def load_translation_dataset(en_path, fr_path):
    """
    Loads parallel data assuming line-by-line alignment between the two files.
    
    """
    if not os.path.exists(en_path):
        raise FileNotFoundError(f"English data file not found at: {en_path}")
    if not os.path.exists(fr_path):
        raise FileNotFoundError(f"French data file not found at: {fr_path}")

    print(f"Reading English file from {en_path}...")
    with open(en_path, 'r', encoding='utf-8') as f:
        en_lines = [line.strip() for line in f]

    print(f"Reading French file from {fr_path}...")
    with open(fr_path, 'r', encoding='utf-8') as f:
        fr_lines = [line.strip() for line in f]

    if len(en_lines) != len(fr_lines):
        raise ValueError(
            f"The number of lines in the English and French files do not match. "
            f"English: {len(en_lines)}, French: {len(fr_lines)}"
        )

    data = []
    print(f"Creating {len(en_lines)} instruction pairs...")
    for i in range(len(en_lines)):
        en_text = en_lines[i]
        fr_text = fr_lines[i]
        
        if en_text and fr_text:
            data.append({
                "instruction": en_text,
                "response": fr_text
            })
            
    return data

In [ ]:
def create_prompt_format(sample):
    """Creates a formatted prompt string from a dataset sample."""
    return f"English:\n{sample['instruction']}\nFrench:\n{sample['response']}"


train_en_path = "/data3/ritika/data/raw/train_100h_txt/train/train.en"
train_fr_path = "/data3/ritika/data/raw/train_100h_txt/train/train.fr"


training_data = load_translation_dataset(train_en_path, train_fr_path)



print(f"Loaded {len(training_data)} samples.")


dataset = Dataset.from_list(training_data)


formatted_dataset = dataset.map(lambda sample: {'text': create_prompt_format(sample)})

Reading English file from /data3/ritika/data/raw/train_100h_txt/train/train.en...
Reading French file from /data3/ritika/data/raw/train_100h_txt/train/train.fr...
Creating 47271 instruction pairs...
Loaded 47271 samples.


Map: 100%|███████████████████████████████████████████████████████████████████| 47271/47271 [00:02<00:00, 20057.28 examples/s]


In [7]:
print(dataset[0])

{'instruction': 'ADIEU VALENTINE ADIEU', 'response': '--Adieu, Valentine, adieu!'}


In [8]:
print(formatted_dataset[0])

{'instruction': 'ADIEU VALENTINE ADIEU', 'response': '--Adieu, Valentine, adieu!', 'text': 'English:\nADIEU VALENTINE ADIEU\nFrench:\n--Adieu, Valentine, adieu!'}


In [ ]:
training_args = TrainingArguments(
    output_dir=f"{output_base_dir}/results_translation",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=1, 
    optim="paged_adamw_32bit",
    save_steps=100,
    save_total_limit=2,
    logging_steps=10, 
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset,
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=128, 
    tokenizer=tokenizer,
    args=training_args,
    packing=False,
)

Map: 100%|████████████████████████████████████████████████████████████████████| 47271/47271 [00:32<00:00, 1449.70 examples/s]


In [10]:
print("Starting training on the custom translation dataset...")
trainer.train(resume_from_checkpoint=True)
print("Training complete.")

Starting training on the custom translation dataset...


/home/siddhi/miniconda3/envs/towerbase-qlora/lib/python3.10/site-packages/transformers/trainer.py:2553: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join

KeyboardInterrupt: 

In [ ]:
final_model_path = f"{output_base_dir}/{new_model_name}"
print(f"Saving fine-tuned model adapter to {final_model_path}")

trainer.model.save_pretrained(final_model_path, safe_serialization=True)
print("Model adapter saved.")